<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# MTBS Fire Data

_by Joe Hamman (CarbonPlan), December 17, 2020_

We have produced a 4km monthly fire data product from the MTBS database.

As background, MTBS itself provides two products: raw fire perimeter shapefiles with individual fire
dates, and an annual-only "burn severity" mosaic that uses LANDAST data to ascribe pixel-level
severity. We combined these to obtain a monthly raster product thresholded by burn severity.

The steps to create this data product were as follows:

- Extract fire perimeters directly from MTBS shape files for all fire categories except for
  prescribed burns
- Create a binary raster for each month by "burning in" shapes for all fires with dates in that
  month
- Mask each raster with the MTBS burn severity mosaic from the corresponding year to include only
  moderate or high severity fires
- Aggregate from 30m to 4km with averaging, treating masked regions in the original data as missing
  values

We independently validated that, in any given year, the sum of this monthly dataset across months
was essentially identical to the annual burn severity mosoaic product restricted to the same
severity thresholds. Any differences could be explained by the fact that the existing annual product
appears to include prescribed burns, whereas we have excluded them.

This notebook loads the 4km rasters and aggregates integrated risk to supersection and ecoregion.
The resulting datasets are stored in GeoJSON for later use.


In [ ]:
import regionmask as rm
import hvplot.pandas
from carbonplan_retro.fire.project_risk import (
    get_baileys,
    get_supersections,
    get_mtbs,
    integrated_risk,
)

First we load the monthly MTBS data and regional polygons (supersections and Bailey's ecoregions).


In [ ]:
prefix = "https://storage.googleapis.com/carbonplan-data"

da = get_mtbs(prefix, load=False).sel(time=slice("2001", "2018")).load()
baileys = get_baileys(prefix)
supersections = get_supersections(prefix)
regions = {"baileys": baileys, "supersections": supersections}

In [ ]:
def to_series(da):
    """helper function to unpack risk data"""
    s = da.to_series()
    s.index = s.index.astype(int)
    return s

## Calculate fire risk and aggregate by region

Now we will loop through the regions and create new GeoDataFrames that include a new column
(`integrated_risk`). We do this by first rasterizing the regional polygons and calculating the
aggregated risk for each polygon.


In [ ]:
out_gdfs = {}

for reg_name, region_gdf in regions.items():
    print(len(region_gdf))
    masks = rm.mask_geopandas(region_gdf, da)
    avg_risk = da.groupby(masks).mean().groupby("time.year").sum().mean("year")
    avg_risk_s = to_series(avg_risk)
    region_gdf["integrated_risk"] = np.nan
    for i, val in avg_risk_s.iteritems():
        region_gdf["integrated_risk"][i] = integrated_risk(val)
    out_gdfs[reg_name] = region_gdf

Now that we have fire risk data for all the polygons in each dataset, we can show it all on a map.
First supersections, then Bailey's ecoregions.


In [ ]:
out_gdfs["supersections"].hvplot(
    c="integrated_risk",
    cmap="YlOrRd",
    hover_cols=["SSection", "integrated_risk"],
)

In [ ]:
out_gdfs["baileys"].hvplot(c="integrated_risk", cmap="YlOrRd", hover_cols=["integrated_risk"])

Finally, we write the expanded GeoDataFrame to GeoJSON files for later use.


In [ ]:
for key, gdf in out_gdfs.items():
    dst_path = f"/Users/jhamman/CarbonPlan Dropbox/Projects/Microsoft/Forests-Retrospective/carbonplan-retro/fire/{key}.json"

    with open(dst_path, "w") as f:
        f.write(gdf.to_json(indent=2))